In [27]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from chat import init_model as init_nanoGPT
from chat import respond as get_respond_nanoGPT

In [13]:
data_path = '../../../data/emotion/validation/100_validation.csv'
df = pd.read_csv(data_path)

In [14]:
df.head()

,Situation,grouped_emotion,empathetic_dialogues,labels
0,Last night I heard strange noises coming from ...,afraid,In the middle of the night I heard some weird ...,Should have grabbed the gun.
1,My mom and sister threw me a baby shower when ...,excited,that was very nice of them congratulations,"Thank you! It was so nice, I had no idea it w..."
2,I just applied for a new job. After the inter...,grateful,"Oh really, do you feel like you did a great job?",I do! I'm feeling very optimistic about it
3,I loaned some money to my friend at work. Turn...,annoyed,Wow! What a jerk for him to up and leave with ...,"It was a medium amount of money but still, he ..."
4,I was out walking late last night and seen som...,NaN,Oh my god. What happened?,"Well, I started walking much faster. It looked..."


## Get model response

In [21]:
init_from = 'withoutemotion/singleConversation'
model = init_nanoGPT(init_from)
def get_response_from_nanoGPT(row):
    situation = row['Situation']
    emotion = row['grouped_emotion']
    human = row['empathetic_dialogues']
    start = '<bot> ' + human + '<human>'
    response, new_emotion, new_context = get_respond_nanoGPT(start, 1, model=model, enable_print=False)
    return response #, new_emotion, new_context

Loading model from: ../trained-saved/withoutemotion/singleConversation/ckpt.pt
number of parameters: 3.42M


/Users/yanxi.lin/Downloads/Project-ML/src/models/nanoGPT/chat.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=device)


In [ ]:
df['new_label'] = df.apply(get_response_from_nanoGPT, axis=1)
# df[['new_label', 'new_emotion', 'new_context']] = df.apply(calculate_new_columns, axis=1, result_type='expand')


In [23]:
df.head()

,Situation,grouped_emotion,empathetic_dialogues,labels,new_label
0,Last night I heard strange noises coming from ...,afraid,In the middle of the night I heard some weird ...,Should have grabbed the gun.,That's great. I hope you do well
1,My mom and sister threw me a baby shower when ...,excited,that was very nice of them congratulations,"Thank you! It was so nice, I had no idea it w...",It was a nice surprise!
2,I just applied for a new job. After the inter...,grateful,"Oh really, do you feel like you did a great job?",I do! I'm feeling very optimistic about it,I don't know. I am so happy for you.
3,I loaned some money to my friend at work. Turn...,annoyed,Wow! What a jerk for him to up and leave with ...,"It was a medium amount of money but still, he ...",That's a bummer. I don't know what to do. Th...
4,I was out walking late last night and seen som...,NaN,Oh my god. What happened?,"Well, I started walking much faster. It looked...",I was driving in the middle of the night and ...


## BLEU

In [28]:
bleu_scores = []
smoothing_function = SmoothingFunction().method1  # To avoid 0 scores due to short sentences
for _, row in df.iterrows():
    for ref, output in zip(row['labels'], row['new_label']):
        # Tokenize each sentence (split by words)
        reference_tokens = [ref.split()]  # BLEU expects a list of lists for references
        output_tokens = output.split()
        
        # Calculate BLEU score
        bleu = sentence_bleu(reference_tokens, output_tokens, smoothing_function=smoothing_function)
        bleu_scores.append(bleu)


In [31]:
average = sum(bleu_scores) / len(bleu_scores)
average

0.005699318863913823